# Input Test Value in Functional Programing with Python

## Origin of the Project

### get_value(): basic function to retrieve input value

In [39]:

def get_value(message):
    '''
    Return the value provided through input()
    
        Parameters:
            - message (str): A message to display for the user
            
        Return:
            _ (str): the value provided by user
    
    '''
    return input(f'{message} :')

### The functions test: testing the input value

In [40]:

def is_safe_not_empty(value):
    '''
    Test is given value is not empty that includes white space.
    Raises ValueError with message 'Please enter information'.
    
        Parameters:
            - value (str): a given value
        
        Return:
            - value (str): the given value
    '''
    
    if not value.strip():
        raise ValueError('Please enter information.')
    else:
        return value


def is_safe_digit(numb_str):
    '''
    Test if a given value can be converted to an integer object.
    
    If value non convertible, raises ValueError with default Error message: "Please provide a int"  
    If value is convertible, return the given value.

    
        Parameters:
            numb_str (str): A given string value
            
        Returns:
            numb_int (int): an interger object
    '''
    try:
        number_int = int(numb_str)
    except ValueError:
        raise ValueError('Please, provide a int.')
    else: 
        
        return number_int
        
    
      
def is_safe_zero_division(numb_int):
    '''
    Test if a give value can be use as denomitor for division. 
    
    If value is zero, raises ZeroDivisionError with default Error message: "Can't divide by 0."  
    If value is not zero, return the value
    
        Parameters:
            - numb_str (int): A given int value
    '''

    try:
        7/int(numb_int)
    except ZeroDivisionError:
        raise ZeroDivisionError('Can\'t divide by 0.')
            
    else:
            return numb_int
        
        



def is_dummy():
    return 'good'

In [41]:

def test_value(test_func, recall_func, value):
    '''
    Test a given value with a given test function. 
    
    If test failed, raises exception with personalised error message and return a recall function. 
    If test succeed, returnes a given value
    
        Parameters:
            - test_func (function): A test function which is going to test the given value. 
            - recall_funct (function): A function that is going to be recalled if test failed (ie. give_value())
            - value: a value that needs to be tested.
    '''
        
    try:
        tested_value=test_func(value)
        
    except Exception as exp:
        print(f'{"Error :":<8} {exp}')
        return recall_func()
    else:
        return value


In [45]:
def get_value_and_test(*funcs, message):
    '''
    Recursive function
    Starting from an input function (ie. get_value()), it will test the return
    value with successive test functions.
    
        Parameters:
            - *functs (tuple): list of functions with successive order from left to right.
            The furthest left must be dedicated to input function (ie. get_value()).
            ie. testing if input value is not empty and convertible to interger object:
            (is_safe_digit, is_safe_not_empty, get_value)
            - message (str): A message to display for the user when input is run.
        
        Returns:
            If value pass the test(s), tested_value_approved (*): input value     
    
    '''
    
    # first level - encapsulate the get get_value and tests

    def launch_get_value():
        
        # second level - run the input() function 
        
        # If one of the test function raises exception - recall the function'launch_get_value' 
        # though the function test_value (see function test_value)
        
        *tests, get_value=funcs # destructure functions, given that the last one must be the input() function
        
        value = get_value(message) # run get value and assign the input to variable value
        
        def launch_test(*tests, value):  
            
            # third level - Recursive function
        
            len_tests = len(tests) 
            
            if len_tests ==1: # last test : end of the recursion 
                
                last_test = tests[0]
                
                tested_value_approved =  test_value(last_test, launch_get_value, value) # if last test raises exeception call launch_get_value
                
                return tested_value_approved # end of get_value_and_test return approved tested_value 
                
            else:
            
                *last_tests,  first_test = tests # get the last test of the list of test. If hierachy between tests (
                                                 # ie. input must resturn an int(str), this test would have to be picked first)
            
                tested_value = test_value(first_test, launch_get_value, value) # if last test raises exeception call launch_get_value

                # return launch_test(last_tests, value=tested_value) if len(last_tests)==1 else launch_test(*last_tests, value=tested_value)
                
                return launch_test(*last_tests, value=tested_value) # recursion of launch_test function wiht -1 function in test function

              
        return launch_test(*tests, value=value) # !!! launch_test FIRST RUN HERE 

    return launch_get_value
  

## Application with the creation of a "Safe" divide function

In [65]:
def divide(x, y):
    return int(x)/int(y)


def safe_divide(function):
    
    def run_safe_divide():
        numerator = get_value_and_test(is_safe_digit,
                                   is_safe_not_empty,
                                   get_value,
                                   message='Please enter an numerator')()
        
        denominator = get_value_and_test(is_safe_zero_division,
                                     is_safe_digit,
                                     is_safe_not_empty,
                                     get_value,
                                     message='Please enter an denominator')()
        
        return function(numerator, denominator)
    
    return run_safe_divide
    
run_safe_divide = safe_divide(divide)



In [66]:
run_safe_divide()

Please enter an numerator : s


Error :  Please, provide a int.


Please enter an numerator : 9
Please enter an denominator : 0


Error :  Can't divide by 0.


Please enter an denominator : 3


3.0

## Extension: other value checking

### tests function

In [16]:
import re

def is_safe_email(email):
    '''
    Test is given value is format email. 
    Require to use re library. 
    Raises Value error is give value is not in the email format.
    
        Parameter:
            - email (str): an given value
            
        Return: 
            - email (str): an given value
    '''
    if not re.fullmatch(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', email):
        raise ValueError('Please enter email')
    else:
        return email
    
    


In [61]:
get_email = get_value_and_test(is_safe_email, is_safe_not_empty, get_value, message='Please enter an email')

get_email()



Please enter an email : sksks


Error :  Please enter email


Please enter an email : clement.liscoet@gmail.com


'clement.liscoet@gmail.com'

In [63]:
def create_arg_checker(arg_check_func, warning_message):
  def arg_checker(func):
    def safe_version(*args, **kwargs):
      if arg_check_func(*args, **kwargs):
        print(f'ARGS: {warning_message}')
        return

      return func(*args, **kwargs)

    return safe_version

  return arg_checker

def second_arg_zero(*args):
  return args[1] == 0;

def first_arg_gt_100(*args):
  return args[0] > 100

@create_arg_checker(second_arg_zero, 'Warning: second arg is zero!')
@create_arg_checker(first_arg_gt_100, 'Warning: first arg is greater than 100')
def divide(x, y):
  return x / y

def name_is_lt_2_chars(*args, **kwargs):
  return len(kwargs['name']) < 2

@create_arg_checker(name_is_lt_2_chars, 'Warning: name arg must be longer than 2 chars')
def create_person(name, age):
  return {
    'name': name,
    'age': age,
  }

